In [1]:
import os 

In [2]:
%pwd 

'c:\\Users\\ahmed\\OneDrive\\Documents\\Documents\\coding\\data science\\DataScience\\MLOPS-Project\\research'

I want to move to my project folder 

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ahmed\\OneDrive\\Documents\\Documents\\coding\\data science\\DataScience\\MLOPS-Project'

## config entity 

this file goes to the src\mlproject\entity\config_entity.py

In [5]:
# this is an entity and we use down on other functions to define the return type

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 



## configurations file

this goes  to the src\mlproject\config\configurations.py

In [6]:
from mlproject.constants import * 
from mlproject.utils.common import read_yaml, create_directories

In [7]:
# class and functions created to make the source files accessible 

class ConfigurationManager():
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig: # creating a function and the return type should be the DataIngestionConfig we created
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir = config.unzip_dir 
        )

        return data_ingestion_config

## data ingestion

this goes to the src\mlproject\components\data_ingestion.py

In [15]:
import os 
import urllib.request as request  # to download data from url
import zipfile # to unzip the data
from mlproject import logger  # if u want to do some logging
from mlproject.utils.common import get_size  # to check size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

# this will create a directory called data ingestion and then it will download the data from given url and the save the log
    def download_file(self): 
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.config.local_data_file))}')
 
    def extract_zip_file(self): # this function will unzip the file and save it in the certain folder
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  

## the pipeline

goes to src\mlproject\pipeline\stage1dataingestion

In [16]:
try:
    config = ConfigurationManager() # initializing the configuration manager
    data_ingestion_config = config.get_data_ingestion_config()  # from configuration manager i am calling the get_data_ingestion config
    data_ingestion = DataIngestion(config=data_ingestion_config) # return the top to the DataIngestion class
    data_ingestion.download_file() # then calling the download function to download the file
    data_ingestion.extract_zip_file() # then unzipping the file
except Exception as e:
    raise e

[2025-09-07 18:34:23,993: INFO: common: yaml file: config\config.yaml, loaded successfully]
[2025-09-07 18:34:23,995: INFO: common: yaml file: params.yaml, loaded successfully]
[2025-09-07 18:34:23,998: INFO: common: yaml file: schema.yaml, loaded successfully]
[2025-09-07 18:34:24,001: INFO: common: created directory at: artifacts]
[2025-09-07 18:34:24,004: INFO: common: created directory at: artifacts/data_ingestion]
[2025-09-07 18:34:24,399: INFO: 578962164: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 175758
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "23237eaa68092cffd4feb41725aae8233503a84e547898e8bfc1b1589e230694"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 41C0:4CB01:99EFEB:D339B3:68BD97A0
Accept-Ranges: bytes
Date: Sun, 